In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor
import  category_encoders as ce
from category_encoders import *

In [2]:
filename = "/home/joaolrossi/Downloads/Finalized_model.sav"
final_model = pickle.load(open(filename,"rb"))

In [3]:
#reading train data to estimate the target encoding
url = "https://raw.githubusercontent.com/joao-l-rossi/LHL-Predicting-Flight-Delays/main/data/Flights_TRAIN.csv"
dataset = pd.read_csv(url)
dataset = dataset.dropna()

In [10]:
#reading test data
url = "https://raw.githubusercontent.com/joao-l-rossi/LHL-Predicting-Flight-Delays/main/data/Flights_TEST.csv"
dataset_TEST = pd.read_csv(url)
dataset_TEST = dataset_TEST.dropna().reset_index(drop=True)
dataset_TEST.columns

Index(['fl_date', 'mkt_carrier', 'mkt_carrier_fl_num', 'origin', 'dest',
       'dep_datetime', 'arr_datetime', 'dep_dayofweek', 'arr_dayofweek',
       'op_unique_carrier', 'tail_num', 'crs_elapsed_time', 'distance',
       'dep_Rain', 'dep_Fog', 'dep_Snow', 'dep_Cold', 'dep_Storm', 'dep_Hail',
       'dep_Precipitation', 'arr_Rain', 'arr_Fog', 'arr_Snow', 'arr_Cold',
       'arr_Storm', 'arr_Hail', 'arr_Precipitation', 'crs_dep_time',
       'crs_arr_time', 'flights_origin', 'flights_dest'],
      dtype='object')

In [47]:
#columns to keep and to encode
cols_to_encode = ["origin", "dest","op_unique_carrier"]
cols_X = ['dep_dayofweek','op_unique_carrier', 'origin', 'dest',
       'arr_Fog', 'arr_Snow', 'arr_Cold', 'arr_Storm', 'arr_Hail',
          'arr_Precipitation', 'crs_dep_time', 'crs_arr_time',
          'flights_origin', 'flights_dest']

In [48]:
# Setting up X and y
X_train = dataset[cols_X]
y_train = dataset["arr_delay"]
X_test = dataset_TEST[cols_X]

In [49]:
#Creating encode object
encoder = TargetEncoder(cols=cols_to_encode)

#Encoding test
encoder.fit(X_train, y_train)
X_test_fitted = encoder.transform(X_test)
X_test_fitted.shape

(148371, 14)

In [50]:
y_pred = final_model.predict(X_test_fitted)
y_pred.shape

(148371,)

In [51]:
submission = pd.concat([dataset_TEST[["fl_date", "mkt_carrier", "mkt_carrier_fl_num", "origin", "dest"]],pd.DataFrame(y_pred) ], axis = 1)

In [55]:
submission.rename(columns = {0:"predicted_delay"}, inplace=True)
submission

,fl_date,mkt_carrier,op_carrier_fl_num,origin,dest,predicted_delay
0,2020-01-01 00:00:00.000,WN,5888,ONT,SFO,6.258602
1,2020-01-01 00:00:00.000,WN,6276,ONT,SFO,5.706332
2,2020-01-01 00:00:00.000,WN,4598,ONT,SJC,9.880496
3,2020-01-01 00:00:00.000,WN,4761,ONT,SJC,6.996217
4,2020-01-01 00:00:00.000,WN,5162,ONT,SJC,0.586618
...,...,...,...,...,...,...
148366,2020-01-07 00:00:00.000,DL,4813,DTW,JFK,-9.933026
148367,2020-01-07 00:00:00.000,DL,4814,GSP,LGA,-12.788722
148368,2020-01-07 00:00:00.000,DL,4815,ATL,XNA,-7.701977
148369,2020-01-07 00:00:00.000,DL,4815,XNA,ATL,-7.566026


In [56]:
submission.to_csv("~/Desktop/LHL-Predicting-Flight-Delays/submission.csv", index=False)